##### Importing pandas and editing display rows settings, and reading the csv file which we extracted

In [1]:
import pandas as pd
import re
pd.set_option('display.max_columns', None)

df = pd.read_csv('training_data.csv')

##### Selecting the additions column to explore it and see if there are any features we can extract from it, thus we group the words and their counts

In [2]:
additions = df['إضافات']
words = additions.str.split(expand=True).stack()
word_frequency = words.value_counts()

word_frequency

مسجل        6075
CD          5825
مُكيّف      5763
مركزي       5667
إغلاق       5603
            ... 
كروزر          1
3000Cc         1
وتحتاج         1
##¥¥.          1
وخاششسسس       1
Name: count, Length: 17869, dtype: int64

##### As we can see above, we a few words have been mentioned more than a thousand time which is a large percentage of the data in our hands, so they probably have an affect on (price)

##### Now we want to select the words that have been mentioned in more than 30% of the total data we have

In [3]:
frequency_threshold = len(df) * 0.3
selected_words = word_frequency[word_frequency > frequency_threshold].index.tolist()
for word in selected_words:
    df[word] = df['إضافات'].apply(lambda x: 1 if word in str(x) else 0)
df = df.drop('إضافات', axis=1)
df
             

,سعر,شركة,موديل,سنة,لون السيارة,نوع الوقود,أصل السيارة,رخصة السيارة,نوع الجير,الزجاج,قوة الماتور,عداد السيارة,عدد الركاب,وسيلة الدفع,معروضة,أصحاب سابقون,مسجل,CD,مُكيّف,مركزي,إغلاق,مغنيسيوم,حماية,هوائية,وسادة,جنطات,جهاز,إنذار,جلد,فرش,سقف,فتحة
0,100000,كيا,اوبتيما,2014,أبيض عاجي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2000,75000,4+1,نقدا فقط,للبيع فقط,يد اولى,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,60000,كيا,سورينتو,2007,سكني,ديزل,خصوصي,فلسطينية,نصف اوتوماتيك,الكتروني,2500,130000,7+1,إمكانية التقسيط,للبيع أو التبديل,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,43500,هونداي,افانتي,2006,سكني,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1600,NaN,NaN,نقدا فقط,للبيع فقط,NaN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1
3,5500,فيات,127,1982,بيج,بنزين,خصوصي,فلسطينية,عادي,يدوي,906,شغال,4+1,إمكانية التقسيط,للبيع فقط,00,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0
4,54000,بيجو,208,2014,فضي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1200,38000,4+1,نقدا فقط,للبيع فقط,NaN,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6859,45000,كيا,مورننغ,2015,فيراني,كهرباء,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1000,130000,4+1,إمكانية التقسيط,للبيع فقط,2,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0
6860,48000,فورد,ترانزيت,2002,أبيض,ديزل,عمومي,فلسطينية,عادي,الكتروني,2400,00000,7+1,نقدا فقط,للبيع فقط,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6861,87000,بيجو,بارتنر,2018,فضي,ديزل,خصوصي,فلسطينية,عادي,الكتروني,1600,50000,4+1,نقدا فقط,للبيع فقط,يد صفر,1,1,1,1,1,0,1,1,1,0,1,1,0,0,0,0
6862,126000,كيا,سورينتو,2017,بترولي,ديزل,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2200,100,6+1,نقدا فقط,للبيع فقط,ثانيه,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


#### So some of the words we extracted are sperated into two columns like "إغلاق مركزي" which are two columns instead of one, which we're going to fix

In [4]:
compound_words_selected = {
    'إغلاق مركزي':['إغلاق','مركزي'],
    'جنطات مغنيسيوم':['جنطات','مغنيسيوم'],
    'وسادة هوائية':['وسادة','هوائية'],
    'جهاز إنذار':['جهاز','إنذار'],
    'فرش جلد':['فرش','جلد'],
    'فتحة سقف':['فتحة','سقف']
}


##### Now we apply the columns edits to the DF

In [5]:
for key, value in compound_words_selected.items():
    df = df.rename(columns={value[0]: key})
    df = df.drop(value[1], axis=1)
df = df.drop('حماية', axis=1)
df

,سعر,شركة,موديل,سنة,لون السيارة,نوع الوقود,أصل السيارة,رخصة السيارة,نوع الجير,الزجاج,قوة الماتور,عداد السيارة,عدد الركاب,وسيلة الدفع,معروضة,أصحاب سابقون,مسجل,CD,مُكيّف,إغلاق مركزي,وسادة هوائية,جنطات مغنيسيوم,جهاز إنذار,فرش جلد,فتحة سقف
0,100000,كيا,اوبتيما,2014,أبيض عاجي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2000,75000,4+1,نقدا فقط,للبيع فقط,يد اولى,1,1,1,1,1,1,1,1,1
1,60000,كيا,سورينتو,2007,سكني,ديزل,خصوصي,فلسطينية,نصف اوتوماتيك,الكتروني,2500,130000,7+1,إمكانية التقسيط,للبيع أو التبديل,2,1,1,1,1,1,1,1,1,1
2,43500,هونداي,افانتي,2006,سكني,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1600,NaN,NaN,نقدا فقط,للبيع فقط,NaN,1,1,1,1,1,1,1,1,1
3,5500,فيات,127,1982,بيج,بنزين,خصوصي,فلسطينية,عادي,يدوي,906,شغال,4+1,إمكانية التقسيط,للبيع فقط,00,1,1,1,0,0,0,0,1,0
4,54000,بيجو,208,2014,فضي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1200,38000,4+1,نقدا فقط,للبيع فقط,NaN,0,0,1,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6859,45000,كيا,مورننغ,2015,فيراني,كهرباء,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1000,130000,4+1,إمكانية التقسيط,للبيع فقط,2,0,0,1,1,1,1,1,0,0
6860,48000,فورد,ترانزيت,2002,أبيض,ديزل,عمومي,فلسطينية,عادي,الكتروني,2400,00000,7+1,نقدا فقط,للبيع فقط,5,1,0,0,0,0,0,0,0,0
6861,87000,بيجو,بارتنر,2018,فضي,ديزل,خصوصي,فلسطينية,عادي,الكتروني,1600,50000,4+1,نقدا فقط,للبيع فقط,يد صفر,1,1,1,1,1,0,1,0,0
6862,126000,كيا,سورينتو,2017,بترولي,ديزل,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2200,100,6+1,نقدا فقط,للبيع فقط,ثانيه,1,1,1,1,1,1,1,1,1


##### Now we want to see what are the catagories for each column, and do some normalization later, i want to see only those that are ambigious

In [6]:
columns = list(df.columns)
columns.remove("سعر")
columns.remove("شركة")
columns.remove("موديل")
columns.remove("سنة")
columns.remove("لون السيارة")
columns.remove("قوة الماتور")

for col in columns:
    print(f'{col} Values: {df[col].unique()}')

نوع الوقود Values: ['بنزين ' 'ديزل ' 'هايبرد ' 'كهرباء ']
أصل السيارة Values: ['خصوصي ' 'عمومي ' 'تأجير ' 'تجاري ' 'تدريب سياقة ' 'حكومي ']
رخصة السيارة Values: ['فلسطينية ' 'نمرة صفراء ']
نوع الجير Values: ['اوتوماتيك ' 'نصف اوتوماتيك ' 'عادي ']
الزجاج Values: ['الكتروني ' 'يدوي ']
عداد السيارة Values: ['75000 ' '130000 ' nan ... '222222 ' '177 ' '7686866 ']
عدد الركاب Values: ['4+1 ' '7+1 ' nan '4+1  ' '5+1 ' '٤+١ ' '8 ' '6+1 ' '4+١ ' '٧ ' '7+ 1 '
 '7:1 ' '1+7 ' '١+٤ ' '2+1 ' '6+١ ' '+1 ' '6±١ ' '22 ' '1+1 ' '٥+١ '
 '٧+1 ' '5+2 ' 'مثال: 4 + 1 ' '7.1 ' '7 =1 ' '٧+١ ' '1+2 ' '6x1     '
 '1+٤ ' '4+1   ' '2/1 ' '8+1 ' '٨+١ ' '٦+١ ' '2 ' '4+2 ' '1٠4 ' '4\\١ '
 '000 ' '3+1 ' '1 ' 'اكثر من 10 ' '9+1 ']
وسيلة الدفع Values: ['نقدا فقط ' 'إمكانية التقسيط ']
معروضة Values: ['للبيع فقط ' 'للبيع أو التبديل ' 'للتبديل فقط ']
أصحاب سابقون Values: ['يد اولى ' '2 ' nan '00  ' '0 ' '1 ' 'يد ثانية ' '4 ' 'يد ثانيه ' '3 '
 'ثالثة ' 'غير منمر استيراد الماني ' '00 ' 'غير منمرة ' 'أولى ' '٨ '
 'يد اولا ' '

##### As seen above some columns are correct and some others have different names for the same meaning and we need to fix that, the columns that beed fixing are 
[
    "عداد السيارة"
    ,"عدد الركاب"
    ,"أصحاب سابقون"
    ]
    The rest seem to be fine,
##### We would start with "عدد الركاب"

##### I can see it has arabic numbers we would start with that first and convert them to english ones, this translation table would be used to translate arabic numbers
##### Second we would want to have them as numbers only so if there is one number keep as is, if two add them up

In [7]:
arabic_numbers = '٠١٢٣٤٥٦٧٨٩'
english_numbers = '0123456789'
translate_arabic_numbers = str.maketrans(arabic_numbers, english_numbers)

def get_number_from_string(string):
   numbers = re.findall(r'\d+', string)
   total = sum(int(num) for num in numbers)
   return total

##### We can place them in a function that we would ultimately use when we transform the "عدد الركاب" column

In [8]:
def extract_num(value):
    if get_number_from_string(str(value).translate(translate_arabic_numbers)):
        return get_number_from_string(str(value).translate(translate_arabic_numbers))
    return 5

df["عدد الركاب"] = df["عدد الركاب"].apply(extract_num)

##### Now we would go with "عداد السيارة" column
##### It has characters other than numbers and there are people who type the numbers without explicitly typing in thousands, .e.g they might type 100 instead of 100000, so these numbers are going to be converted to thousands

In [9]:
def convert_car_meter(value):

    number = re.findall(r'\d+', str(value))

    if(not number):
        return None
    number = number[0]
    
    if (float(number) / 1000) >=1:
        return int(number)
    
    return int(number)*1000

df["عداد السيارة"] = df["عداد السيارة"].apply(convert_car_meter)

##### Now we come for the "أصحاب سابقون" column which needs a bit of workarounds, and i think imma use a dict that has strings which point to a number representing them using a regex

In [10]:
regex_mapping = {
    "شرك":1,
    "صفر":0,
    "اول":1,
    "أول":1,
    "ثان":2,
    "تان":2,
    "ثال":3,
    "تال":3,
    "راب":4,
    "خام":5,
    "ساد":6,
    "ساب":7,
    "ثام":8,
    "تام":8,
    "تاس":9,
    "عاش":10,
    "مست":1,
    "غير":0,
    "مش":0,
    "نفس":1,
}
def previous_users_transform(text):
    string = str(text).translate(translate_arabic_numbers)
    for key,value in regex_mapping.items():
        match = re.search(key,string)
        if(match):
            return value
    if get_number_from_string(string) > -1:
        return get_number_from_string(string)
    return None


In [11]:
df["أصحاب سابقون"] = df["أصحاب سابقون"].apply(previous_users_transform)

#### Now we create a new improved version of the data after the transformations we've done to use it in the next step

In [12]:
df.to_csv("improved_training_data.csv")